In [2]:
import requests
import altair as alt
import pandas as pd
from pandas.io.json import json_normalize

from pyvis.network import Network
from ipywidgets import interact, fixed

import re

%load_ext autoreload
%autoreload 2
%cd ../intervals
from main import *
from main_objs import *
import visualizations as viz

/Users/dangtrang/OneDrive - brynmawr.edu/summer 2021/crim_intervals/intervals


## Relationships heatmaps

In [244]:
df_relationships = pd.read_csv('../CRIM_Intervals_Notebooks/relationships.csv')
files = df_relationships['model_observation.piece.piece_id'].unique()
df_relationships_0011 = df_relationships[df_relationships['model_observation.piece.piece_id'] == files[0]].copy()

In [245]:
df_relationships_0011.head()

,Unnamed: 0,url,id,relationship_type,musical_type,rt_q,rt_q_x,rt_q_monnayage,rt_tm,rt_tm_snd,...,model_observation.piece.url,model_observation.piece.piece_id,model_observation.piece.full_title,model_observation.ema,derivative_observation.url,derivative_observation.id,derivative_observation.piece.url,derivative_observation.piece.piece_id,derivative_observation.piece.full_title,derivative_observation.ema
0,0,http://crimproject.org/data/relationships/1/,1,Quotation,Fuga,True,True,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"1-6/1,1,1-2,1-2,2,2/@1,@1-3,@1-3+@1,@1-3+@1-3,...",http://crimproject.org/data/observations/2/,2,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"1-6/1,1,1+3,1+3,3,3/@1-4,@1-3,@1-3+@1-4,@1-3+@..."
1,1,http://crimproject.org/data/relationships/2/,2,Non-mechanical transformation,Fuga,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"5-9/1,1,1-2,2,2/@3,@1-4,@1-2+@4,@1-4,@1-2",http://crimproject.org/data/observations/4/,4,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"5-8/1,1+4,1+4,4/@3,@1-4+@4,@1+@1-3,@1"
2,2,http://crimproject.org/data/relationships/3/,3,Non-mechanical transformation,Fuga,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"5-9,14-18/1,1,1-2,1-2,1-2,3,3,3-4,3-4,3-4/@3,@...",http://crimproject.org/data/observations/6/,6,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_5,Missa Tota pulchra es: Agnus Dei,"9-12/3-4,2-4,2,2/@4+@1-4,@4+@1-4+@1-3,@1-3,@1"
3,3,http://crimproject.org/data/relationships/4/,4,Non-mechanical transformation,Fuga,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"5-9/1,1,1-2,2,2/@3,@1-4,@1-2+@4,@1-4,@1-2",http://crimproject.org/data/observations/8/,8,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"5-10/1,1+4,1+4,1+4,1+4,4/@3,@1-4+@4,@1-4.5+@1-..."
183,183,http://crimproject.org/data/relationships/184/,184,Non-mechanical transformation,Imitative duo,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"1-7,10-16/1,1,1-2,1-2,2,2,2,3,3,3-4,3-4,4,4,4/...",http://crimproject.org/data/observations/368/,368,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"1-6,10-13/1,1,1+3,1+3,3,3,2,2-3,2-3,3/@1-4,@1-..."


In [246]:
df_relationships_0011[df_relationships_0011['relationship_type']== "Quotation, Mechanical transformation"]

,Unnamed: 0,url,id,relationship_type,musical_type,rt_q,rt_q_x,rt_q_monnayage,rt_tm,rt_tm_snd,...,model_observation.piece.url,model_observation.piece.piece_id,model_observation.piece.full_title,model_observation.ema,derivative_observation.url,derivative_observation.id,derivative_observation.piece.url,derivative_observation.piece.piece_id,derivative_observation.piece.full_title,derivative_observation.ema
256,256,http://crimproject.org/data/relationships/258/,258,"Quotation, Mechanical transformation",Fuga,True,False,True,True,True,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"76-77/1+3,1+3/@3-4.5+@4,@1-2+@1-2.5",http://crimproject.org/data/observations/516/,516,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"85-87/1,1-2,2/@4,@1-3+@2-4.5,@1"


In [247]:
df_relationships_0011.iloc[0, :]

Unnamed: 0                                                                                 0
url                                             http://crimproject.org/data/relationships/1/
id                                                                                         1
relationship_type                                                                  Quotation
musical_type                                                                            Fuga
rt_q                                                                                    True
rt_q_x                                                                                  True
rt_q_monnayage                                                                         False
rt_tm                                                                                  False
rt_tm_snd                                                                              False
rt_tm_minv                                                            

In [248]:
viz.plot_comparison_heatmap(df_relationships_0011, 'model_observation.ema', main_category='musical_type', other_category='observer.name', heat_map_width=800,
                              heat_map_height=300)

alt.VConcatChart(...)

In [249]:
#retrieve_notes_and_durations('https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_3.0/CRIM_Model_0011.mei', '1-6/1,1,1-2,1-2,2,2/@1,@1-3,@1-3+@1,@1-3+@1-3,@1-3,@1')

## Observations heatmap

In [250]:
df_observations = pd.read_csv('../CRIM_Intervals_Notebooks/observations.csv')

In [251]:
df_observations_pen = df_observations[df_observations['musical_type']=='Periodic entry'].copy()
df_observations_pen.iloc[0, :]

Unnamed: 0                                                          5
url                       http://crimproject.org/data/observations/6/
id                                                                  6
ema                     9-12/3-4,2-4,2,2/@4+@1-4,@4+@1-4+@1-3,@1-3,@1
musical_type                                           Periodic entry
                                          ...                        
observer.name                                            Drew Ivarson
piece.url           http://crimproject.org/data/pieces/CRIM_Mass_0...
piece.piece_id                                       CRIM_Mass_0009_5
piece.full_title                     Missa Tota pulchra es: Agnus Dei
piece.mass                                             CRIM_Mass_0009
Name: 5, Length: 92, dtype: object

In [252]:
networks, interactive_df = viz.create_comparisons_networks_and_interactive_df(df_observations, 'mt_fg_int', 'melodic', 'ema')

interactive(children=(Text(value='Input search pattern', description='search_pattern_starts_with'), Output()),…

In [253]:
#networks['5-'].show('../CRIM_Intervals_Notebooks/test.html')

## Ngrams heatmaps

In [4]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0017.mei'])
model = corpus.scores[0]
mel = model.getMelodic(kind='d', compound=False, unit=0)
mel_ngrams = model.getNgrams(df=mel, n=5)
mel_ngrams.head()

Successfully imported.


,[Discantus],[Contratenor],[Tenor],[QuintaVox],[SextaVox],[Bassus]
4.0,NaN,"1, 2, 2, 2, -4",NaN,NaN,NaN,NaN
7.0,NaN,"2, 2, 2, -4, 8",NaN,NaN,NaN,NaN
8.0,"1, 2, 1, 2, -3","2, 2, -4, 8, -2",NaN,NaN,NaN,NaN
10.0,NaN,"2, -4, 8, -2, -2",NaN,NaN,NaN,NaN
12.0,NaN,"-4, 8, -2, -2, 2",NaN,NaN,NaN,NaN


### heatmap

In [255]:
dur1 = model.getDuration(df=mel, n=5, mask_df=mel_ngrams)

In [256]:
viz.plot_ngrams_heatmap(mel_ngrams,ngrams_duration=dur1)

alt.VConcatChart(...)

### close match

In [257]:
mel_ngram_list = mel_ngrams.stack().dropna().value_counts().head(10).index.to_list()

In [258]:
viz.plot_close_match_heatmap(mel_ngrams, "-2, -3, 2, 2, 1", selected_patterns=mel_ngram_list, voices=[])

alt.VConcatChart(...)

In [259]:
viz.plot_close_match_heatmap(mel_ngrams, "1, 2, 1, -3", selected_patterns=mel_ngram_list, voices=[])

alt.VConcatChart(...)

## Weighted Levenshtein

In [5]:
ngrams = model.getNgrams(df=model.getMelodic(kind='d', directed=True, compound=True, unit=0), n=5)
ngrams_dur = model.getDuration(df=model.getMelodic(kind='d', directed=True, compound=True, unit=0), n=5, mask_df=ngrams)
viz.plot_close_match_heatmap(ngrams, '-2, 2, 2, 2, 2', algorithm='w', ngrams_duration=ngrams_dur, interval_type='d')

alt.VConcatChart(...)

In [8]:
ngrams = model.getNgrams(df=model.getMelodic(kind='d', directed=True, compound=True, unit=0), n=-1)
ngrams_dur = model.getDuration(df=model.getMelodic(kind='d', directed=True, compound=True, unit=0), n=-1, mask_df=ngrams)
viz.plot_close_match_heatmap(ngrams, '2, -3, 1, 1, -2, -2, 2, 2, 2, 2, -2, -2, 1, -2, 2', algorithm='w', ngrams_duration=ngrams_dur, interval_type='d')

alt.VConcatChart(...)

## Networks for relationships

In [262]:
df_relationships.iloc[26, :]

Unnamed: 0                                                                                26
url                                            http://crimproject.org/data/relationships/27/
id                                                                                        27
relationship_type                                   Quotation, Non-mechanical transformation
musical_type                                                                Counter-soggetto
rt_q                                                                                    True
rt_q_x                                                                                  True
rt_q_monnayage                                                                         False
rt_tm                                                                                  False
rt_tm_snd                                                                              False
rt_tm_minv                                                            

In [ ]:
nt = viz.plot_relationship_network(df_relationships, selected_model_ids=['CRIM_Model_0008'], color='derivative')
# %cd ../CRIM_Intervals_Notebooks
nt.show('../CRIM_Intervals_Notebooks/0008.html')

In [264]:
df_relationships['model_observation.piece.piece_id'].unique()

array(['CRIM_Model_0011', 'CRIM_Model_0018', 'CRIM_Model_0017',
       'CRIM_Mass_0015_2', 'CRIM_Model_0001', 'CRIM_Model_0020',
       'CRIM_Model_0009', 'CRIM_Model_0019', 'CRIM_Model_0010',
       'CRIM_Model_0022', 'CRIM_Model_0002', 'CRIM_Model_0016',
       'CRIM_Model_0024', 'CRIM_Model_0021', 'CRIM_Mass_0018_1',
       'CRIM_Model_0008', 'CRIM_Mass_0018_3', 'CRIM_Model_0012',
       'CRIM_Model_0023', 'CRIM_Model_0015', 'CRIM_Model_0013',
       'CRIM_Mass_0005_1', 'CRIM_Mass_0016_2', 'CRIM_Mass_0020_3',
       'CRIM_Model_0006', 'CRIM_Model_0025'], dtype=object)

In [265]:
dir(viz)

['Fraction',
 'Network',
 'NormalizedLevenshtein',
 'NormalizedWeightedIntervalLevenshtein',
 'RELATIONSHIP_WEIGHTS',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_close_match',
 '_close_match_helper',
 '_from_ema_to_offsets',
 '_manipulate_processed_network_df',
 '_plot_ngrams_df_heatmap',
 '_process_crim_json_url',
 '_process_ngrams_df_helper',
 '_trim_and_combine_piece_ids_with_measures',
 'alt',
 'create_bar_chart',
 'create_comparisons_networks_and_interactive_df',
 'create_heatmap',
 'create_interactive_compare_df',
 'create_interval_networks',
 'fixed',
 'generate_ngrams_and_duration',
 'group_observations',
 'interact',
 'pd',
 'plot_close_match_heatmap',
 'plot_comparison_heatmap',
 'plot_ngram_network',
 'plot_ngrams_heatmap',
 'plot_pieces_relationship_network',
 'plot_relationship_network',
 'process_network_df',
 'process_ngrams_df',
 're',
 'textdistance']

In [266]:
nt2 = viz.plot_relationship_network(df_relationships, selected_derivative_ids=['CRIM_Mass_0017_3', 'CRIM_Mass_0008_4',
       'CRIM_Mass_0003_5', 'CRIM_Mass_0014_4', 'CRIM_Mass_0018_1',
       'CRIM_Mass_0018_2', 'CRIM_Mass_0005_5'], color='derivative')
nt2.show('../CRIM_Intervals_Notebooks/selected_family.html')

In [267]:
nt3 = viz.plot_relationship_network(df_relationships, selected_model_ids = ['CRIM_Model_0002', 'CRIM_Model_0016'], selected_families=['CRIM_Mass_0017_3', 'CRIM_Mass_0008_4',
       'CRIM_Mass_0003_5', 'CRIM_Mass_0014_4', 'CRIM_Mass_0018_1',
       'CRIM_Mass_0018_2', 'CRIM_Mass_0005_5'], color='derivative')
nt3.show('../CRIM_Intervals_Notebooks/selected_model_family.html')

5 CRIM_Mass_0017_3, CRIM_Mass_0008_4, CRIM_Mass_0018_1, CRIM_Mass_0018_2, CRIM_Mass_0005_5 no longer exist in df because of other filtering options


In [268]:
nt4 = viz.plot_relationship_network(df_relationships, selected_model_ids = ['CRIM_Model_0002', 'CRIM_Model_0016'], color='derivative')
nt4.show('../CRIM_Intervals_Notebooks/selected_model.html')

In [269]:
nt3 = viz.plot_relationship_network(df_relationships, selected_model_ids = ['CRIM_Model_0002'], selected_derivative_ids=['CRIM_Mass_0003_5', 'CRIM_Mass_0014'], selected_families=['CRIM_Mass_0017_3', 'CRIM_Mass_0008_4',
       'CRIM_Mass_0003_5', 'CRIM_Mass_0014_4', 'CRIM_Mass_0018_1',
       'CRIM_Mass_0018_2', 'CRIM_Mass_0005_5'], color='derivative')
nt3.show('../CRIM_Intervals_Notebooks/selected_model_deriv_family.html')

6 CRIM_Mass_0017_3, CRIM_Mass_0008_4, CRIM_Mass_0014_4, CRIM_Mass_0018_1, CRIM_Mass_0018_2, CRIM_Mass_0005_5 no longer exist in df because of other filtering options


## Observations network

In [270]:
df_observations.columns

Index(['Unnamed: 0', 'url', 'id', 'ema', 'musical_type', 'mt_cf',
       'mt_cf_voices', 'mt_cf_dur', 'mt_cf_mel', 'mt_sog', 'mt_sog_voices',
       'mt_sog_dur', 'mt_sog_mel', 'mt_sog_ostinato', 'mt_sog_periodic',
       'mt_csog', 'mt_csog_voices', 'mt_csog_dur', 'mt_csog_mel', 'mt_cd',
       'mt_cd_voices', 'mt_fg', 'mt_fg_voices', 'mt_fg_int', 'mt_fg_tint',
       'mt_fg_periodic', 'mt_fg_strict', 'mt_fg_flexed', 'mt_fg_sequential',
       'mt_fg_inverted', 'mt_fg_retrograde', 'mt_pe', 'mt_pe_voices',
       'mt_pe_int', 'mt_pe_tint', 'mt_pe_strict', 'mt_pe_flexed', 'mt_pe_flt',
       'mt_pe_sequential', 'mt_pe_added', 'mt_pe_invertible', 'mt_id',
       'mt_id_voices', 'mt_id_int', 'mt_id_tint', 'mt_id_strict',
       'mt_id_flexed', 'mt_id_flt', 'mt_id_invertible', 'mt_nid',
       'mt_nid_voices', 'mt_nid_int', 'mt_nid_tint', 'mt_nid_strict',
       'mt_nid_flexed', 'mt_nid_flt', 'mt_nid_sequential', 'mt_nid_invertible',
       'mt_hr', 'mt_hr_voices', 'mt_hr_simple', 'mt_hr_s

In [271]:
#process df
df = df_observations[df_observations['piece.piece_id'] == 'CRIM_Model_0008']
df = df[['ema', 'piece.piece_id',
        'musical_type']]
df['ema'] = df['ema'].str.split("/", n=1, expand=True)[0]
df['piece_part'] = df['piece.piece_id'] + ":" + df['ema']
df

,ema,piece.piece_id,musical_type,piece_part
1761,54-65,CRIM_Model_0008,Imitative duo,CRIM_Model_0008:54-65
1763,1-10,CRIM_Model_0008,Periodic entry,CRIM_Model_0008:1-10
1961,1,CRIM_Model_0008,NaN,CRIM_Model_0008:1
1963,44-46,CRIM_Model_0008,Soggetto,CRIM_Model_0008:44-46
1965,46-48,CRIM_Model_0008,"Fuga, Interval patterns",CRIM_Model_0008:46-48
...,...,...,...,...
5185,40-44,CRIM_Model_0008,Homorhythm,CRIM_Model_0008:40-44
5187,94-110,CRIM_Model_0008,Soggetto,CRIM_Model_0008:94-110
5189,44-48,CRIM_Model_0008,Fuga,CRIM_Model_0008:44-48
5214,1,CRIM_Model_0008,Cantus firmus,CRIM_Model_0008:1


In [272]:
# piece being center, nodes == piece part, musical color = musical type
nt = Network(directed=True, notebook=True)
nt.add_nodes(df['piece.piece_id'])

for index in df['piece_part'].index:
    nt.add_node(df['piece_part'].loc[index], group=df['musical_type'].loc[index])
    
for row in df.index:
    nt.add_edge(df['piece.piece_id'].loc[row], df['piece_part'].loc[row],
                 title=df['musical_type'].loc[row])
    
    
nt.inherit_edge_colors("to")
nt.force_atlas_2based()
nt.show('../CRIM_Intervals_Notebooks/piece_network.html')

## Networks of pieces

In [273]:
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx

df = viz._trim_and_combine_piece_ids_with_measures(df_relationships)

# add the nodes and edges into graphs
G = nx.MultiGraph()
G.add_nodes_from(df['model_observation.piece.piece_id'])
G.add_nodes_from(df['derivative_observation.piece.piece_id'])

for row in df.index:
    G.add_edge(df['model_observation.piece.piece_id'].loc[row], 
              df['derivative_observation.piece.piece_id'].loc[row])

# compute the best partition
partition = community_louvain.best_partition(G)

for node in partition:
    G.nodes[node]['group'] = partition[node]

#convert to pyvis
nt = Network(notebook=True)
nt.from_nx(G)
nt.show('../CRIM_Intervals_Notebooks/community.html')

## Ngrams across pieces' idea

In [274]:
FILES_MANY = [
    'https://crimproject.org/mei/CRIM_Model_0001.mei',
    'https://crimproject.org/mei/CRIM_Model_0002.mei',
#     'https://crimproject.org/mei/CRIM_Model_0008.mei'
#     'https://crimproject.org/mei/CRIM_Model_0009.mei',
    'https://crimproject.org/mei/CRIM_Model_0010.mei',
    'https://crimproject.org/mei/CRIM_Model_0011.mei',
    'https://crimproject.org/mei/CRIM_Model_0012.mei',
    'https://crimproject.org/mei/CRIM_Model_0013.mei',
    'https://crimproject.org/mei/CRIM_Model_0014.mei',
    'https://crimproject.org/mei/CRIM_Model_0015.mei',
    'https://crimproject.org/mei/CRIM_Model_0016.mei',
    'https://crimproject.org/mei/CRIM_Model_0017.mei',
    'https://crimproject.org/mei/CRIM_Model_0019.mei',
    'https://crimproject.org/mei/CRIM_Model_0020.mei',
    'https://crimproject.org/mei/CRIM_Model_0021.mei',
    'https://crimproject.org/mei/CRIM_Mass_0001_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0001_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0001_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0001_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0001_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0002_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0002_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0002_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0002_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0002_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0003_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0003_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0003_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0003_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0003_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0004_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0004_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0004_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0004_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0004_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0005_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0005_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0005_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0005_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0005_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0006_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0006_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0006_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0006_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0006_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0007_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0007_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0007_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0007_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0007_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0008_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0008_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0008_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0008_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0008_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0009_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0009_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0009_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0009_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0009_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0010_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0010_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0010_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0010_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0010_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0011_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0011_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0011_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0011_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0011_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0012_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0012_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0012_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0012_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0012_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0013_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0013_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0013_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0013_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0013_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0014_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0014_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0014_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0014_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0014_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0015_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0015_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0015_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0015_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0015_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0016_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0016_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0016_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0016_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0016_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0017_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0017_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0017_3.mei',
    'https://crimproject.org/mei/CRIM_Mass_0017_4.mei',
    'https://crimproject.org/mei/CRIM_Mass_0017_5.mei',
    'https://crimproject.org/mei/CRIM_Mass_0018_1.mei',
    'https://crimproject.org/mei/CRIM_Mass_0018_2.mei',
    'https://crimproject.org/mei/CRIM_Mass_0018_3.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0018_4.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0018_5.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0019_1.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0019_2.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0019_3.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0019_4.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0019_5.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0020_1.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0020_2.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0020_3.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0020_4.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0020_5.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0021_1.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0021_2.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0021_3.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0021_4.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0021_5.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0022_1.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0022_2.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0022_3.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0022_4.mei',
#     'https://crimproject.org/mei/CRIM_Mass_0022_5.mei'
]

In [ ]:
# #{ngram: name}
# ngrams_dict = {}
# corpus = CorpusBase(FILES_MANY)

In [ ]:
# ans = []
# def process_df(ngram, piece_name):
#     df = pd.DataFrame(ngram).reset_index()
#     df = df.rename(columns={'index': 'pattern', 0:'count'})
#     df['piece'] = piece_name
#     return df

# for index in range(len(FILES_MANY)):
#     name = FILES_MANY[index].split("/")[-1]
#     model =  corpus.scores[index]
#     ngram = model.getNgrams(df=model.getMelodic(kind='d', compound=False, unit=0), n=5).stack().value_counts()
#     ans.append(process_df(ngram, name))

In [ ]:
# df = pd.concat(ans, axis=0)
# df.reset_index(drop=True, inplace=True)
# df.sort_values(inplace=True, by='count', ascending=False)
# top_pattern = df.head(20)['pattern'].to_list()
# df = df[df['pattern'].isin(top_pattern)].dropna()

In [ ]:
# G2 = nx.Graph()
# net = Network(height='600px', width='600px', notebook=True)

# for row in df.index:
#     G2.add_node(df['piece'].loc[row])
#     G2.add_node(df['pattern'].loc[row])
#     G2.add_edge(df['piece'].loc[row], df['pattern'].loc[row], 
#                 weight=int(df['count'].loc[row]))
    
# # compute the best partition
# partition2 = community_louvain.best_partition(G2)

# print(len(corpus.scores))

# for node in partition2:
#     G2.nodes[node]['group'] = partition2[node]
    
# net.from_nx(G2)
# # # net.barnes_hut()
# # # net.force_atlas_2based()
# # net.repulsion(node_distance=50, central_gravity=0.1, spring_length=200, spring_strength=0.05, damping=0.01)
# net.show('../CRIM_Intervals_Notebooks/ngrams_community_network.html')

In [ ]:
# edges